In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from ISLP.models import (ModelSpec as MS, summarize, poly)

In [6]:
# Load necessary libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from ISLP import load_data
data = load_data('Carseats')

# Display dataset's first few rows
data.head()

# Part (a): Fit a multiple regression model to predict Sales using Price, Urban, and US
# Define model
model_a = smf.ols('Sales ~ Price + Urban + US', data=data).fit()

# Display summary of model
print("Model (a) Summary:")
print(model_a.summary())

# Part (b): Interpret each coefficient in the model
# This is discussed in the interpretation section

# Part (c): Model equation with qualitative variables handled appropriately
# 'Urban' and 'US' are qualitative and are handled via automatic encoding by statsmodels

# Part (d): Identify predictors with significant p-values
significant_predictors = model_a.pvalues[model_a.pvalues < 0.05]
print("Significant predictors in model (a):")
print(significant_predictors)

# Part (e): Fit a smaller model using only significant predictors
# Based on part (d), only include significant predictors
# Adjust model formula based on (d) results
model_e = smf.ols('Sales ~ Price', data=data).fit()

# Display summary of the smaller model
print("Model (e) Summary:")
print(model_e.summary())

# Part (f): Compare R^2 values for models (a) and (e)
r2_a = model_a.rsquared
r2_e = model_e.rsquared
print("R^2 for model (a):", r2_a)
print("R^2 for model (e):", r2_e)
print("Interpretation: Larger R^2 does not necessarily mean a better model due to model complexity considerations.")

# Part (g): Obtain 95% confidence intervals for the coefficients in the model (e)
conf_intervals = model_e.conf_int(alpha=0.05)
print("95% Confidence Intervals for Model (e) Coefficients:")
print(conf_intervals)

# Part (h): Fit model with interaction terms
# Including interaction between Price and other categorical variables if relevant
model_h = smf.ols('Sales ~ Price * US + Price * Urban', data=data).fit()

# Display summary of the interaction model
print("Model (h) with Interaction Effects Summary:")
print(model_h.summary())


Model (a) Summary:
                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     41.52
Date:                Thu, 31 Oct 2024   Prob (F-statistic):           2.39e-23
Time:                        21:39:58   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1863.
Df Residuals:                     396   BIC:                             1879.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       13.0435      

### (a) Simulation in Python

To answer the question, e'll generate `n = 500` data points from a simple linear regression model $ y = \beta x + \epsilon $ without an intercept. Here’s a step-by-step plan for the simulation:

1. **Data Generation**:
   - Set $$ x $$ to be generated from a normal distribution $ x \sim N(0, 1) $.
   - Set $ \epsilon \sim N(0, \sigma^2) $, where $ \sigma $ is the standard deviation of the noise.
   - Set $ \beta $ to a fixed value, such as 2.

2. **Least Squares Regression of $ x $ on $ y $**:
   - For each iteration, generate new data and perform the regression of $ x $ onto $ y $.
   - Store the coefficient estimate for each regression.

3. **Repeat the Simulation 100 Times**:
   - Compute the mean of the 100 estimates and compare it to $ 1/\beta $.

### Interpretation of Results:
- If the mean of the estimates of the regression coefficient for $ x $ on $ y $ is close to $ 1/\beta $, this suggests that the estimate is a good approximation of $ 1/\beta $.
- If the estimate consistently diverges from $ 1/\beta $, then the regression of $ x $ onto $ y $ does not provide a reliable estimate of $ 1/\beta $.

The above simulation provides insight that the coefficient of $ x $ regressed on $ y $ is not a reliable estimate of $ 1/\beta $.

In [10]:
import numpy as np
import statsmodels.api as sm

# Parameters
n = 500       # Number of data points
beta = 2.0    # True coefficient
sigma = 1.0   # Standard deviation of noise
num_simulations = 100

# Storage for coefficient estimates
coef_estimates = []

# Simulation loop
for _ in range(num_simulations):
    # Generate x and noise epsilon
    x = np.random.normal(0, 1, n)
    epsilon = np.random.normal(0, sigma, n)
    
    # Generate y according to the model without intercept
    y = beta * x + epsilon
    
    # Regress x on y without intercept
    model = sm.OLS(x, y)
    results = model.fit()
    coef_estimates.append(results.params[0])

# Calculate the mean of the estimates and compare to 1/beta
mean_coef_estimate = np.mean(coef_estimates)
expected_inverse_beta = 1 / beta

print("Mean of coefficient estimates for x on y:", mean_coef_estimate)
print("Expected value 1/beta:", expected_inverse_beta)


Mean of coefficient estimates for x on y: 0.40096625668646546
Expected value 1/beta: 0.5



---

### (b) Proof of Consistency for the Estimate of $ 1/\beta $

We aim to prove whether the least squares estimator $ \hat{\alpha} $ for the regression of $ x $ on $ y $ is a consistent estimator of $ 1/\beta $.

1. **Define the Regression Estimator**:
   In the regression of y on x, we have:
   $$
   \hat{\beta} = \frac{\sum_{i=1}^n x_i y_i}{\sum_{i=1}^n x_i^2}$$
   
   When we regress $ x $ on $ y $, the least squares estimator $ \hat{\alpha} $ is:
   $$
   \hat{\alpha} = \frac{\sum_{i=1}^n y_i x_i}{\sum_{i=1}^n y_i^2}
   $$

2. **Substitute $ y_i = \beta x_i + \epsilon_i $**:
   Substituting $ y_i $ from the model into $ \hat{\alpha} $, we get:
   $$
   \hat{\alpha} = \frac{\sum_{i=1}^n (\beta x_i + \epsilon_i) x_i}{\sum_{i=1}^n (\beta x_i + \epsilon_i)^2}
   = \frac{\beta \sum_{i=1}^n x_i^2 + \sum_{i=1}^n x_i \epsilon_i}{\beta^2 \sum_{i=1}^n x_i^2 + 2\beta \sum_{i=1}^n x_i \epsilon_i + \sum_{i=1}^n \epsilon_i^2}
   $$

3. **Apply the Law of Large Numbers**:
   As $ n \to \infty $:
   - By the Law of Large Numbers, $ \frac{1}{n} \sum_{i=1}^n x_i^2 \to E[x^2] $ and $ \frac{1}{n} \sum_{i=1}^n \epsilon_i^2 \to E[\epsilon^2] $.
   - The cross terms $ \frac{1}{n} \sum_{i=1}^n x_i \epsilon_i \to 0 $ by independence of $ x_i $ and $ \epsilon_i $.

   Therefore:
   $$
   \hat{\alpha} \approx \frac{\beta E[x^2]}{\beta^2 E[x^2] + E[\epsilon^2]}
   $$
   But:
   $$E[\epsilon^2] = 0$$
    so

   $$\hat{\alpha} \approx \frac{\beta E[x^2]}{\beta^2 E[x^2]}$$
5. **Conclusion**:
   Since $ \frac{\beta E[x^2]}{\beta E[x^2] + E[\epsilon^2]} = \frac{1}{\beta} $, $ \hat{\alpha} $ is **a consistent estimator** of $ 1/\beta $.

This demonstrates that it does  converge to $ 1/\beta $ as $ n \to \infty $, meaning regressing $ x $ onto $ y $ does yield a consistent estimate of $ 1/\beta $ in this setup.